In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mne
from tqdm import tqdm
from moabb.datasets import BNCI2014001
from mne.time_frequency import tfr_multitaper, tfr_morlet
from matplotlib.colors import TwoSlopeNorm
import pandas as pd
import seaborn as sns

### Load BCI IV / A dataset
Find the dataset description [here](https://lampz.tugraz.at/~bci/database/001-2014/description.pdf) and [here](https://neurotechx.github.io/moabb/generated/moabb.datasets.BNCI2014001.html#moabb.datasets.BNCI2014001)

In [ ]:
ds = BNCI2014001()
data = ds.get_data()

### Data exploration

In [ ]:
raw = data[1]['session_T']['run_0']
stims, stim_t = raw[raw.ch_names.index('stim')]
unique_events = np.unique(stims, return_counts=True)  # 0 == no event

In [ ]:
# create events and epochs; check doc for timing
tmin, tmax = .8, 4
baseline = tmin, 2
events = mne.find_events(raw, 'stim')
event_id = dict(left=1, right=2, feet=3, tongue=4)
epochs = mne.Epochs(raw, events, event_id, tmin=tmin, tmax=tmax, baseline=baseline, preload=True)

In [ ]:
# plot individual epochs
epochs.plot(scalings=.0001, events=events)

# within-condition epochs averaging
for avg in epochs.average(by_event_type=True):
    avg.plot(titles=avg.comment, hline=[0])

In [ ]:
# check power spectral density
epochs.plot_psd()

### Epoching

In [ ]:
# TODO implement function to: re-reference electrodes, bandpass filter (0.5 - 100Hz) and notch filter (50, 100Hz), then apply baselinin
def prep_epoch(raw: mne.io.Raw):
    pass
    # return epochs

In [ ]:
# combine epochs
subjects = range(1, 2)  # TODO use more and more subjects
all_epochs = []
for subj_i, subj in data.items():
    if subj_i not in subjects:
        continue
    for raw in tqdm(subj['session_T'].values(), desc=str(subj_i)):
        epochs = prep_epoch(raw.copy())
        all_epochs.append(epochs)

all_epochs = mne.concatenate_epochs(all_epochs)

### Time-frequency analysis

In [ ]:
# TODO try both tfr_morlet and tfr_multitaper to generate time-frequency signal
#   see differences in the generated spectrograms
# TODO your time-freq plot should show alpha and beta frequencies,
#   and the C3, Cz and C4 channels only

# TODO try different freqs and n_cycles parameters

### Alpha ERDS analysis

In [ ]:
# TODO compute alpha and beta ERDS using the above computed time-freq signals,
#   show left and right ERDS on 3 electrodes: C3, Cz, C4